In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'prepare/mesolitica-tpu.json'

from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-tpu-general')

In [2]:
from glob import glob
import itertools
from tqdm import tqdm
import json

with open('../summary/translated-paraphrase.json') as fopen:
    data = json.load(fopen)

data = [i[1] for i in data]

len(data)

62592

In [3]:
data[:5]

['Ketua pegawai operasi PCCW, Mike Butcher, dan Alex Arena, ketua pegawai kewangan, akan melapor terus kepada Encik So. <> Ketua Pegawai Operasi Semasa Mike Butcher dan Ketua Pegawai Kewangan Kumpulan Alex Arena akan melapor kepada So.',
 'Dua pembuat kenderaan terbesar di dunia mengatakan penjualan A.S. mereka menurun lebih banyak daripada yang diramalkan bulan lalu kerana kegilaan penjualan akhir musim panas menyebabkan lebih banyak reaksi industri daripada yang dijangkakan. <> Penjualan domestik di GM dan No. 2 Ford Motor Co. merosot lebih banyak daripada yang diramalkan kerana kegilaan penjualan akhir musim panas mendorong reaksi industri yang lebih besar daripada yang dijangkakan.',
 'Menurut Pusat Kawalan dan Pencegahan Penyakit persekutuan (berita - laman web), terdapat 19 kes campak yang dilaporkan di Amerika Syarikat pada tahun 2002. <> Pusat Kawalan dan Pencegahan Penyakit mengatakan terdapat 19 kes campak yang dilaporkan di Amerika Syarikat pada tahun 2002.',
 'Syarikat peny

In [4]:
data[0].split(' <> ')

['Ketua pegawai operasi PCCW, Mike Butcher, dan Alex Arena, ketua pegawai kewangan, akan melapor terus kepada Encik So.',
 'Ketua Pegawai Operasi Semasa Mike Butcher dan Ketua Pegawai Kewangan Kumpulan Alex Arena akan melapor kepada So.']

In [5]:
x = 'Ahli bidaah harus dikucilkan, dan kemudian diserahkan kepada kelompok sekular, yang akan menjatuhkan hukuman yang sewajarnya kepada mereka (animadversio debita). Ekskomunikasi hanyalah permulaan hukuman mereka, kerana mereka kemudian diserahkan kepada pemerintah sekular atas apa yang disebut animadverio debita, atau layak dihukum.'
x.split('.')

['Ahli bidaah harus dikucilkan, dan kemudian diserahkan kepada kelompok sekular, yang akan menjatuhkan hukuman yang sewajarnya kepada mereka (animadversio debita)',
 ' Ekskomunikasi hanyalah permulaan hukuman mereka, kerana mereka kemudian diserahkan kepada pemerintah sekular atas apa yang disebut animadverio debita, atau layak dihukum',
 '']

In [10]:
import re

before, after = [], []

for row in data:
    splitted = row.split('<>')
    if len(splitted) != 2:
        if ',.' in row:
            splitted = row.split(',.')
        elif '< >' in row:
            splitted = row.split('< >')
        elif '<' in row:
            splitted = row.split('<')
        elif '>' in row:
            splitted = row.split('>')
        elif '.' in row:
            splitted = row.split('.')
            splitted = [i for i in splitted if len(i) > 2]
            if len(splitted) != 2:
                print('titik', row, '\n')
                continue 
        
        else:
            print('else', row, '\n')
            continue
    
    l = re.sub(r'[ ]+', ' ', splitted[0]).strip()
    r = re.sub(r'[ ]+', ' ', splitted[1]).strip()
    
    if len(splitted[1]) < 10:
        continue
        
    if l == r:
        continue
        
    if len(l) > 10 and len(r) > 10:
        
        before.append(l)
        after.append(r)
        before.append(r)
        after.append(l)

titik Pihak berkuasa mengatakan saintis itu dikarantina dengan betul di rumah setelah dia mengalami gejala SARS pada 10 Disember. 

titik Ketua dan CEO Oracle Corp Larry Ellison tidak menolak mengesahkan tawaran syarikat yang tidak diminta untuk memperoleh pesaing PeopleSoft Inc. 

titik instrumen naik 4us1.37 ke 4us19.25 dan sistem novellus maju 4us1.76 ke 4us36.31, masing-masing dinaikkan menjadi kelebihan berat badan 'oleh lehman. 

titik Senator Richard Shelby, R-Ala., Ketua Jawatankuasa Perbankan Senat, tidak tergesa-gesa untuk mengesahkan undang-undang tersebut. Senator Richard Shelby, seorang Republikan dari Alabama, ketua Jawatankuasa Perbankan Senat, tidak tergesa-gesa untuk mengesahkan undang-undang itu, kata jurucakap jawatankuasa itu Andrew Gray. 

titik Kes itu keluar dari Illinois dan melibatkan syarikat untung bernama Telemarketing Associates Inc. 

titik Bersama dengan pembuat chip Intel, syarikat-syarikat tersebut termasuk Sony Corp., Microsoft Corp., Hewlett-Packard C

titik Olahragawan dan rakan sejawat Wallace meletakkan Jimmy Smith pada 1909 "All American Team." 

titik Semasa penjajahan Portugis dan Brazil, pengaruh Portugis juga ditambahkan, dan Feijoada dimasukkan ke wilayah Guisos yang lain. Pengaruh Portugis juga ditambahkan semasa penjajahan Portugis dan Brazil. Feijoada dimasukkan ke dalam guisos yang lain. 

titik Jack Cross adalah siri buku komik yang ditulis oleh Gary Erskine dan dilukis oleh Warren Ellis. Ia pertama kali diterbitkan oleh DC Comics pada tahun 2005. Jack Jack Cross adalah siri komik yang ditulis oleh Gary Erskine dan dilukis oleh Warren Ellis. Ia pertama kali diterbitkan oleh DC Comics pada tahun 2005. 

else Dalam teori komputer - kerumitan, masa polinomial secara rawak (RP) adalah kelas kerumitan masalah yang mana mesin turing probabilistik wujud dengan ciri-ciri ini: mesin Turing kebarangkalian wujud dengan sifat-sifat ini: 

titik Para saintis yang disebutkan di atas, Adam, adalah: Pythagoras, Galileo Galilei, Nicolau

In [11]:
len(before), len(after)

(117430, 117430)

In [8]:
from sklearn.model_selection import train_test_split

before, test_before, after, test_after = train_test_split(before, after, test_size = 1000)

In [11]:
with open('../summary/paraphrase-set.json', 'w') as fopen:
    json.dump({'before': before, 'test_before': test_before,
              'after': after, 'test_after': test_after}, fopen)

In [12]:
batches = []
batch = 30000
for i in range(0, len(before), batch):
    index = min(i + batch, len(before))
    x = before[i: index]
    y = after[i: index]
    batches.append((x, y))

In [13]:
import tensorflow as tf

for i in range(len(batches)):
    before = batches[i][0]
    after = batches[i][1]
    filename = f'bahasa-paraphrase-{i}.tsv'
    with tf.io.gfile.GFile(filename, 'w') as outfile:
        for i in range(len(before)):
            outfile.write('%s\t%s\n' % (before[i], after[i]))
            
    blob = bucket.blob(f't5-data/{filename}')
    blob.upload_from_filename(filename)